In [1]:
import spacy
from spacy.training.example import Example
import pickle
import random

ModuleNotFoundError: No module named 'spacy'

In [2]:
train_data = pickle.load(open('train_data.pkl', 'rb'))
train_data

NameError: name 'pickle' is not defined

In [41]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner', last=True)
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

    #Prepare the data

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):     #Only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting iteration "+str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0

            
            for text, annotation in train_data:
                try:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotation)
                    nlp.update(
                        [example],   #batch of Example objects
                        drop = 0.2,
                        sgd = optimizer,
                        losses = losses
                    )
                #     nlp.update(
                #         [text],   #Batch of texts
                #         [annotation],   #Batch of annotations
                #         drop = 0.2,
                #         sgd = optimizer,
                #         losses = losses
                #     )
                except Exception as e:
                    #print(e)
                    pass
                
            print('losses')
            print(losses)

In [42]:
#Training the model
train_model(train_data)

Starting iteration 0
losses
{'ner': 10057.4903923769}
Starting iteration 1
losses
{'ner': 4319.248714557079}
Starting iteration 2
losses
{'ner': 2544.4841815113823}
Starting iteration 3
losses
{'ner': 2146.6085002281175}
Starting iteration 4
losses
{'ner': 1900.3484439596375}
Starting iteration 5
losses
{'ner': 1751.7195115033749}
Starting iteration 6
losses
{'ner': 1599.9575738060448}
Starting iteration 7
losses
{'ner': 1563.4968174207386}
Starting iteration 8
losses
{'ner': 1286.2523714916606}
Starting iteration 9
losses
{'ner': 1239.8304078584551}


In [43]:
#Storing the trained NLP model

nlp.to_disk('nlp_model')

In [46]:
#Load the trained nlp model
nlp_model = spacy.load('nlp_model')

In [47]:
train_data[0][0]

"Mohini Gupta Server Support Engineer  Gurgaon, Haryana - Email me on Indeed: indeed.com/r/Mohini-Gupta/08b5b8e1acd8cf07  Willing to relocate: Anywhere  WORK EXPERIENCE  Server Support Engineer  Microsoft -  July 2015 to November 2017  Key Responsibilities: ● Worked as Technical Support Engineer for Microsoft Enterprise Platforms Support.  ● Worked in U.S., U.K., India & APAC time zones.  ● Always available (24X7) for any explanation, support or information required by team, client and managers.  ● Configuring, deploying and troubleshooting SCCM with remote/local SQL. ● Perform Software distribution and ensure successful deployment on end assets. ● Patch management of servers and end user estate along with troubleshooting. ● Performed on checks through server, to perform server operations, check services, analyzed the logs to check the communication from the new server to the primary server and vice versa, also checked server's communication with its client. ● Setting up new packages a

In [48]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Mohini Gupta
DESIGNATION                   - Server Support Engineer
EMAIL ADDRESS                 - indeed.com/r/Mohini-Gupta/08b5b8e1acd8cf07
DESIGNATION                   - Server Support Engineer
COMPANIES WORKED AT           - Microsoft
DEGREE                        - B.tech
COLLEGE NAME                  - KIIT college of Engg.
SKILLS                        - active directory, iis, sccm, dhcp, sql, wsus, dns  ADDITIONAL INFORMATION  Computer Skills ● MS Office Tools: MS Excel, MS Word, MS Power Point. ● Hands on experience on all versions of Windows. ● Sound knowledge of internet and networking. ● Coding Languages: C, C++, Java.
LOCATION                      - Gurgaon


## Now run the test resumes to check whether it is working or not

In [1]:
#Convert PDF to text using PyMuPDF, as PyPDF2 does not work properly
import sys
import fitz
fname = "Smith Resume.pdf"
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())

print(text)

Michael Smith 
BI / Big Data/ Azure 
Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f 
 
10+ years of Experience in Designing, Development, Administration, Analysis, 
Management 
inthe 
Business 
Intelligence 
Data 
warehousing, 
Client 
Server 
Technologies, Web-based Applications, cloud solutions and Databases. 
Data warehouse: Data analysis, star/ snow flake schema data modeling and design 
specific todata warehousing and business intelligence environment. 
Database: Experience in database designing, scalability, back-up and recovery, 
writing andoptimizing SQL code and Stored Procedures, creating functions, views, 
triggers and indexes.  
Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL 
Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure 
data lake analytics(U-SQL). 
Big Data: Worked Azure data lake store/analytics for big data processing and Azure 
data factoryto schedule U-SQL jobs. Designed and d

In [60]:
#Modifing the extracted text
tx = " ".join(text.split('\n'))   #New line will replaced with black space
tx

'Michael Smith  BI / Big Data/ Azure  Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f    10+ years of Experience in Designing, Development, Administration, Analysis,  Management  inthe  Business  Intelligence  Data  warehousing,  Client  Server  Technologies, Web-based Applications, cloud solutions and Databases.  Data warehouse: Data analysis, star/ snow flake schema data modeling and design  specific todata warehousing and business intelligence environment.  Database: Experience in database designing, scalability, back-up and recovery,  writing andoptimizing SQL code and Stored Procedures, creating functions, views,  triggers and indexes.   Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL  Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure  data lake analytics(U-SQL).  Big Data: Worked Azure data lake store/analytics for big data processing and Azure  data factoryto schedule U-SQL jobs. Designed and 

In [61]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Michael Smith
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - The University of Manchester
SKILLS                        - problem solving (Less than 1 year), project lifecycle (Less than 1 year), project  manager (Less than 1 year), technical assistance. (Less than 1 year)  ADDITIONAL INFORMATION  Professional Skills
